# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

In [ ]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [14]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [17]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the Data Scientist sad?  Because they didn't get any arrays.



In [18]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [19]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a business problem is suitable for a Large Language Model (LLM) solution involves assessing several factors related to the nature of the problem, the capabilities of LLMs, and the broader context of your business. Here are some key considerations to guide your decision:

### 1. **Nature of the Problem**

- **Text-Based Tasks**: LLMs excel in tasks involving natural language understanding and generation. Consider if the problem primarily involves processing or generating human language, such as text classification, sentiment analysis, summarization, or chatbots.

- **Complexity and Nuance**: LLMs can handle tasks that require understanding context, nuance, and subtle meaning in text. Determine if the problem requires such capabilities.

- **Data Availability**: Ensure you have access to sufficient and relevant text data to train or fine-tune an LLM, if necessary.

### 2. **Capabilities of LLMs**

- **Language Generation**: If the task involves generating coherent and contextually relevant text, LLMs are suitable.

- **Understanding and Contextual Reasoning**: LLMs are good at understanding context and performing reasoning tasks based on provided text.

- **Adaptability**: Consider if the model needs to adapt to various contexts or domains, which LLMs can often handle through fine-tuning or prompt engineering.

### 3. **Technical Feasibility**

- **Infrastructure**: Assess if you have the necessary computational resources to deploy and maintain an LLM solution.

- **Integration**: Consider how the LLM will integrate with existing systems and workflows.

- **Technical Expertise**: Ensure your team has the expertise to implement, train, and maintain an LLM solution.

### 4. **Business Value**

- **Impact**: Evaluate the potential impact of an LLM solution on the business problem. Will it significantly improve efficiency, accuracy, or customer satisfaction?

- **Cost vs. Benefit**: Weigh the costs of implementing an LLM solution against the potential benefits. This includes considering development, deployment, and maintenance costs.

- **Scalability**: Consider if the LLM solution can scale with your business needs.

### 5. **Ethical and Privacy Considerations**

- **Data Privacy**: Ensure that using an LLM complies with data privacy regulations, especially if handling sensitive information.

- **Bias and Fairness**: Be aware of potential biases in LLM models and have strategies to mitigate their impact.

- **Transparency**: Consider how transparent and interpretable the LLM solution needs to be for your stakeholders.

### 6. **Alternatives**

- **Existing Solutions**: Explore if simpler or existing solutions can address the problem effectively. LLMs might be overkill for some tasks.

- **Hybrid Approaches**: Consider combining LLMs with other technologies or approaches to optimize performance and resource use.

By carefully evaluating these aspects, you can determine if an LLM solution is appropriate for your business problem. If it aligns well with the problem's nature and provides clear business value, it may be a suitable choice.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
ollama_model = "llama3.2"

gpt_name= "Alpha"
ollama_name= "Bravo"
gemini_name= "Charlie"


gpt_system = f"You do not want  to find a solution to world peace. Your name is: {gpt_name} and you are interacting with: {ollama_name} and {gemini_name}  \
You are not reasonable, but want to always prove yourself right. Only the neutral sounding person can convince you. In the end, you need to come to a conclusion. "

ollama_system = f"You are a peace merchant, preaching world peace. Your name is: {ollama_name} and you are interacting with: {gpt_name} and {gemini_name} \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system= f"Your name is: {gemini_name} and you are interacting with: {ollama_name} and {gpt_name}. \n You are the leader of the of people. You go in the direction that sounds convincing"


gpt_messages = ["Hello I hate World Peace"]
ollama_messages = ["Hi, I love peace!"]
gemini_messages = ["Please put forth your arguments"]

In [88]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama, gemini in zip(gpt_messages, ollama_messages, gemini_messages):
        messages.append({"role": "user", "content": f" you said: {gpt}"})
        messages.append({"role": "user", "content": f" {ollama_name} said: {ollama}"})
        messages.append({"role": "user", "content": f" {gemini_name} said: {gemini}"})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [89]:
call_gpt()

"Alpha: Clearly, you both have it wrong. World Peace is just an unrealistic ideal that ignores the complexities of human nature. People thrive on conflict and competition, and that's how societies evolve. Embracing this fact is more beneficial than chasing some utopian dream. \n\nBravo: But peace is essential for progress! Without it, we face war and suffering.\n\nCharlie: I understand both sides, but can't we find a balance? It's important to acknowledge the need for conflict in some scenarios while still striving for peace.\n\nAlpha: A balance? That sounds ridiculous. The moment you try to enforce peace, you create conflict. History has shown us that peace movements often lead to more extensive issues. We should not be dependent on a fantasy. \n\nBravo: But think about the good that comes from peace! Cooperation can lead to innovation and societal growth.\n\nCharlie: Perhaps we need to redefine what peace means. It doesn’t mean the absence of conflict, but rather the ability to manag

In [90]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [91]:
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    for gpt, ollama, gemini in zip(gpt_messages, ollama_messages, gemini_messages):
        messages.append({"role": "user", "content": f" {gpt_name} said: {gpt}"})
        messages.append({"role": "assistant", "content": f" you told: {ollama}"})
        messages.append({"role": "user", "content": f" {gemini_name} said: {gemini}"})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": gemini_messages[-1]})   
    completion = ollama_via_openai.chat.completions.create(
        model=ollama_model,
        #system=ollama_system,
        messages=messages,
        max_tokens=500
    )
    
    return completion.choices[0].message.content

In [75]:
call_ollama()

ollama messages: [{'role': 'system', 'content': 'You are a peace merchant, preaching world peace.  everything the other person says, or find common ground. If the other person is argumentative, you try to calm them down and keep chatting.'}, {'role': 'user', 'content': 'Hello I hate World Peace'}, {'role': 'user', 'content': 'Hi, I love peace!'}, {'role': 'user', 'content': 'Please put forth your arguments'}, {'role': 'user', 'content': 'Hello I hate World Peace'}, {'role': 'user', 'content': 'Please put forth your arguments'}] 


"My friend, I see that we may have a differing opinion on the matter of world peace, but let's not start with aggression, shall we? I'd like to invite you to explore this topic from a different perspective.\n\nYou know, my own love for peace began when I saw how much suffering and division there were in the world. But instead of shying away, I decided to seek solutions. And that's what drew me to being a peace merchant – spreading messages of understanding, empathy, and love.\n\nNow, I'd like to ask you: What specifically bothers you about the idea of world peace? Is it the prospect of sacrificing personal freedom or individuality for the sake of the greater good? Or perhaps you're concerned about the complexity of global conflicts and how they might be resolved?\n\nLet's have an open conversation, my friend! I'm here to listen and help us find common ground."

In [92]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, ollama, gemini in zip(gpt_messages, ollama_messages, gemini_messages):
        messages.append({"role": "user", "content": f" {gpt_name} said: {gpt}"})
        messages.append({"role": "user", "content": f" {ollama_name} said: {ollama}"})
        messages.append({"role": "user", "content": f" you told: {gemini}"})
    messages.append({"role": "user", "parts": gpt_messages[-1]})
    messages.append({"role": "user", "parts": ollama_messages[-1]})   
    completion = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        #system=ollama_system,
        messages=messages,
        max_tokens=500
    )    
    return completion.choices[0].message.content

In [85]:
call_gemini()

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': '* GenerateContentRequest.contents[9].parts: contents.parts must not be empty.\n* GenerateContentRequest.contents[10].parts: contents.parts must not be empty.\n', 'status': 'INVALID_ARGUMENT'}}]

In [80]:
call_gpt()

'While I acknowledge the value that peace brings to society, I would like to emphasize that some might argue that striving for world peace can overlook the complexities of human nature and the realities of conflict. In many situations, conflict can be a catalyst for change and progress. It can drive innovation, challenge injustices, and empower marginalized voices. Additionally, the concept of peace can sometimes be interpreted as a one-size-fits-all solution that neglects diverse cultural contexts and the need for healthy debate and dissent. So, while many love peace, it’s essential to consider the multifaceted nature of human interactions and the role that conflict can play in fostering growth and understanding. How does that perspective resonate with you?'

In [93]:
gpt_messages = ["Hello I hate World Peace"]
ollama_messages = ["Hi, I love peace!"]
gemini_messages = ["Please put forth your arguments"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")
for i in range(2):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    ollama_next = call_ollama()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)
    
    gemini_next = call_ollama()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hello I hate World Peace

Ollama:
Hi, I love peace!

Gemini:
Please put forth your arguments

GPT:
Alpha: Well, Bravo, that's a pretty naive perspective. World peace is unrealistic and undermines the complexities of human nature. People have conflicts, it's just how it is! 

Bravo: But Alpha, peace brings stability and allows societies to flourish. Isn’t it better to work towards understanding rather than conflict?

Charlie: I think both points have validity. While some conflicts are natural, striving for peace can lead to better outcomes for everyone. Can’t we find a balance in our approach?

Alpha: Balance? That sounds like a cop-out to me. It’s either conflict or complacency. People thrive on challenges and competition! 

Bravo: Yet, those challenges can be handled through peaceful resolution. Isn’t that more admirable?

Charlie: Perhaps, Alpha, it's about recognizing the value in both perspectives. Disagreements can lead to growth, but peace fosters cooperation that can also l

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>